# Parte 9 - Intro a Programas Encriptados

Lo crean o no, es posible hacer cálculos con datos encriptados. En otras palabras, es posible ejecutar un programa donde **TODAS las variables** están **encriptadas**!

En este tutorial, vamos a ir paso a paso por las herramientas más básicas de la computación encriptada. Específicamente, veremos un enfoque popular llamado Computación Segura Multi-Parte (Secure Multi-Party Computation). En esta lección, aprenderemos cómo construir una calculadora encriptada que puede hacer operaciones en números encriptados. 

Autores:
- Andrew Trask - Twitter: [@iamtrask](https://twitter.com/iamtrask)
- Théo Ryffel - GitHub: [@LaRiffle](https://github.com/LaRiffle)

Referencias: 
- Morten Dahl - [Blog](https://mortendahl.github.io) - Twitter: [@mortendahlcs](https://twitter.com/mortendahlcs)

Traductores:
- Daniel Firebanks-Quevedo - GitHub: [@thefirebanks](https://www.github.com/thefirebanks)

# Paso 1: Encriptación Usando Computación Segura Multi-Parte (CSMP)

A primera vista, CSMP es una manera extraña de "encriptación". En vez de usar una clave pública/privada para encriptar una variable, cada valor es dividido en múltiples `acciones`, cada una funcionando como una clave privada. Típicamente, éstas `acciones` serán distribuidas entre 2 o más dueños. Por lo tanto, para descifrar la variable, todos los dueños deben de estar de acuerdo en permitir el desciframiento. En esencia, cada persona tiene una clave privada. 

### Encrypt()

Digamos que queremos "encriptar" una variable `x`, podríamos hacerlo de la siguiente manera.

 > La encriptación no usa decimales o números reales, sino que ocurre en un espacio matemático llamado [anillo cociente](https://es.wikipedia.org/wiki/Anillo_cociente), el cual abarca los números enteros entre `0` y `Q-1`, donde `Q` es un número primo y "suficientemente grande" para que el espacio pueda contener todos los números que usamos en nuestros experimentos. En la práctica, dado un número entero `x`, nosotros hacemos `x % Q` para encajar en el anillo. (Esa es la razón por la que evitamos usar un número `x' > Q`).

In [ ]:
Q = 1234567891011

In [ ]:
x = 25

In [ ]:
import random

def encrypt(x):
    share_a = random.randint(-Q,Q)
    share_b = random.randint(-Q,Q)
    share_c = (x - share_a - share_b) % Q
    return (share_a, share_b,  share_c)

In [ ]:
encrypt(x)

Como pueden ver aquí, hemos dividido nuestra variable `x` en 3 diferentes acciones, las cuales pueden ser enviadas a 3 dueños diferentes.

### Decrypt()

Si quisiéramos descifrar estas 3 acciones, podríamos simplemente sumarlas y tomar el módulo del resultado (mod Q)

In [ ]:
def decrypt(*shares):
    return sum(shares) % Q

In [ ]:
a,b,c = encrypt(25)

In [ ]:
decrypt(a, b, c)

Nótese que si tratamos de descifrar el valor de `x` con solo dos acciones, el descifrado no funciona!

In [ ]:
decrypt(a, b)

Por lo tanto, necesitamos a todos los dueños para poder hacer el descifrado. De esta manera, las `acciones` sirven como claves privadas, las cuales tienen que estar presente para descifrar un valor.

# Paso 2: Aritmética Básica Utilizando CSMP

Sin embargo, la propiedad de la Computación Segura Multi-Parte que es realmente extraordinaria es la habilidad para hacer computaciones **mientras las variables están encriptadas**. Vamos a demostrar una simple adición.

In [ ]:
x = encrypt(25)
y = encrypt(5)

In [ ]:
def add(x, y):
    z = list()
    # el primer trabajador suma sus acciones
    z.append((x[0] + y[0]) % Q)
    
    # el segundo trabajador suma sus acciones
    z.append((x[1] + y[1]) % Q)
    
    # el tercer trabajador suma sus acciones
    z.append((x[2] + y[2]) % Q)
    
    return z

In [ ]:
decrypt(*add(x,y))

### Éxito!!!

Y aquí lo tienen! Si cada trabajador (de manera separada) suma sus acciones, las acciones resultantes van a descifrar el valor correcto (25 + 5 == 30).

Resulta que existen protocolos de CSMP que permiten esta computación encriptada para las siguientes operaciones:
- adición (que acabamos de ver)
- multiplicación
- comparación

y utilizando estas operaciones primitivas, podemos hacer computaciones arbitrarias!!

En la siguiente sección, vamos a aprender a cómo usar la librería PySyft para hacer tales operaciones!

# Paso 3: CSMP Usando PySyft

En las secciones anteriores, describimos la intuición básica sobre como la CSMP debe de funcionar. Sin embargo, en la práctica no queremos escribir todas las operaciones primitivas nosotros cuando escribimos nuestros programas encriptados. En esta sección vamos a aprender lo básico sobre de la computación encriptada usando PySyft. Específicamente, nos vamos a enfocar en cómo ejecutar los 3 primitivos antes mencionados: adición, multiplicación y comparación.

Primero, necesitamos crear unos cuantos Trabajadores Virtuales (con los cuales esperemos que estén ya familiarizados dados nuestros tutoriales previos)

In [ ]:
import torch
import syft as sy
hook = sy.TorchHook(torch)

bob = sy.VirtualWorker(hook, id="bob")
alice = sy.VirtualWorker(hook, id="alice")
bill = sy.VirtualWorker(hook, id="bill")

### Encripción/Descifrado Básico

La encripción es tan simple como tomar cualquier tensor de PySyft y llamar .share(). El descifrado es tan simple como llamar .get() en la variable compartida.

In [ ]:
x = torch.tensor([25])

In [ ]:
x

In [ ]:
encrypted_x = x.share(bob, alice, bill)

In [ ]:
encrypted_x.get()

### Introspección de los Valores Encriptados

Si vemos de cerca a los trabajadores de Bob, Alice y Bill podemos ver las acciones que se crean!

In [ ]:
bob._objects

In [ ]:
x = torch.tensor([25]).share(bob, alice, bill)

In [ ]:
# Las acciones de Bob
bobs_share = list(bob._objects.values())[0]
bobs_share

In [ ]:
# Las acciones de Alice
alices_share = list(alice._objects.values())[0]
alices_share

In [ ]:
# Las acciones de Bill
bills_share = list(bill._objects.values())[0]
bills_share

Y si quisiéramos, podríamos descifrar estos valores utilizando EL MISMO enfoque del que hablamos antes!

In [ ]:
Q = x.child.field

(bobs_share + alices_share + bills_share) % Q

Como pueden ver, cuando llamamos `.share()`, simplemente partió el valor en 3 acciones y envió una acción a cada una de las partes!

# Aritmética Encriptada

Ahora pueden ver que podemos hacer aritmética con los valores! La API está construida de tal manera que podemos hacer aritmética como haríamos con los tensores de Pytorch.

In [ ]:
x = torch.tensor([25]).share(bob,alice)
y = torch.tensor([5]).share(bob,alice)

In [ ]:
z = x + y
z.get()

In [ ]:
z = x - y
z.get()

# Multiplicación Encriptada

Para la multiplicación necesitamos una parte adicional, quien es responsable de generar números aleatorios constantemente (y que no colude con ninguna de las otras partes). Denominamos a esta persona la "proveedora de encripción". Para nuestros propósitos, la proveedora de encripción es solo una VirtualWorker (Trabajadora Virtual) adicional, pero es importante reconocer que la proveedora no es una "dueña", en el sentido que ella no posee acciones propias pero es alguien en quien confiamos que no coludirá con los dueños existentes de las acciones.

In [ ]:
crypto_provider = sy.VirtualWorker(hook, id="crypto_provider")

In [ ]:
x = torch.tensor([25]).share(bob,alice, crypto_provider=crypto_provider)
y = torch.tensor([5]).share(bob,alice, crypto_provider=crypto_provider)

In [ ]:
# multiplicación

z = x * y
z.get()

También pueden hacer multiplicación de matrices

In [ ]:
x = torch.tensor([[1, 2],[3,4]]).share(bob,alice, crypto_provider=crypto_provider)
y = torch.tensor([[2, 0],[0,2]]).share(bob,alice, crypto_provider=crypto_provider)

In [ ]:
# multiplicación de matrices

z = x.mm(y)
z.get()

# Comparación Encriptada

Es también posible el hacer comparaciones privadas entre valores privados. En este caso contamos con el protocolo SecureNN, cuyos detalles pueden encontrar [aquí](https://eprint.iacr.org/2018/442.pdf). El resultado de la comparación es también un tensor compartido privado.

In [ ]:
x = torch.tensor([25]).share(bob,alice, crypto_provider=crypto_provider)
y = torch.tensor([5]).share(bob,alice, crypto_provider=crypto_provider)

In [ ]:
z = x > y
z.get()

In [ ]:
z = x <= y
z.get()

In [ ]:
z = x == y
z.get()

In [ ]:
z = x == y + 20
z.get()

También pueden hacer operaciones de max.

In [ ]:
x = torch.tensor([2, 3, 4, 1]).share(bob,alice, crypto_provider=crypto_provider)
x.max().get()

In [ ]:
x = torch.tensor([[2, 3], [4, 1]]).share(bob,alice, crypto_provider=crypto_provider)
max_values, max_ids = x.max(dim=0)
max_values.get()

# !Felicitaciones! - !Es hora de unirte a la comunidad!

¡Felicitaciones por completar esta parte del tutorial! Si te gustó y quieres unirte al movimiento para preservar la privacidad, propiedad descentralizada de IA y la cadena de suministro de IA (datos), puedes hacerlo de las siguientes formas!

### Dale una estrella a PySyft en GitHub

La forma más fácil de ayudar a nuestra comunidad es por darle estrellas a los repositorios de Github! Esto ayuda a crear consciencia de las interesantes herramientas que estamos construyendo.

- [Dale una estrella a PySyft](https://github.com/OpenMined/PySyft)

### ¡Únete a nuestro Slack!

La mejor manera de mantenerte actualizado con los últimos avances es ¡unirte a la comunidad! Tú lo puedes hacer llenando el formulario en [http://slack.openmined.org](http://slack.openmined.org)

### ¡Únete a un proyecto de código!

La mejor manera de contribuir a nuestra comunidad es convertirte en un ¡contribuidor de código! En cualquier momento puedes ir al Github Issues de PySyft y filtrar por "Proyectos". Esto mostrará todos los tiquetes de nivel superior dando un resumen de los proyectos a los que ¡te puedes unir! Si no te quieres unir a un proyecto, pero quieres hacer un poco de código, también puedes mirar más mini-proyectos "de una persona" buscando por Github Issues con la etiqueta "good first issue".

- [Proyectos de PySyft](https://github.com/OpenMined/PySyft/issues?q=is%3Aopen+is%3Aissue+label%3AProject)
- [Good First Issue Tickets](https://github.com/OpenMined/PySyft/issues?q=is%3Aopen+is%3Aissue+label%3A%22good+first+issue%22)

### Donar

Si no tienes tiempo para contribuir a nuestra base de código, pero quieres ofrecer tu ayuda, también puedes aportar a nuestro Open Collective". Todas las donaciones van a nuestro web hosting y otros gastos de nuestra comunidad como ¡hackathons y meetups!

[OpenMined's Open Collective Page](https://opencollective.com/openmined)